In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import numpy as np 
import matplotlib.pyplot as plt
import json
from PIL import Image

### Model in Tensorflow for (128x128x3) Input

In [ ]:
test_tensor = tf.random.normal([1,128,128,3])
test_tensor = tf.cast(test_tensor, tf.float32)

In [ ]:
##dummy model 
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(2)
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        print(x.shape)

        return self.d2(x)

# Create an instance of the model
model = MyModel()

In [ ]:
result_128 = model(test_tensor)

In [ ]:
model.save('/Users/prakanshulsaxena/Downloads/temp_model')

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('/Users/prakanshulsaxena/Downloads/temp_model/') # path to the SavedModel directory
tflite_model = converter.convert()

In [ ]:
with open('/Users/prakanshulsaxena/Downloads/model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
interpreter = tf.lite.Interpreter('/Users/prakanshulsaxena/Downloads/model.tflite')

In [ ]:
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
interpreter.set_tensor(input_details[0]['index'], test_tensor)
interpreter.invoke()

In [ ]:
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

#### Function 
#### Input-> Image Path and Path of TFLite Model
#### Output-> X and Y Coordinates

In [ ]:
IMAGE_LOCATION = 
MODEL_LOCATION = 

In [ ]:
def query_gmodel(image_path,model_path):
    interpreter = tf.lite.Interpreter(model_path)
    
    image = Image.open(image_path)
    image = np.asarray(image)
    image_t = tf.convert_to_tensor(image)
    image_t = tf.expand_dims(image_t, 0)
    
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], test_tensor)
    interpreter.invoke()b
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    return output_data    

In [ ]:
out_test_1 = query_gmodel(image_path,model_path)
print(out_test_1)

#### Function
#### Input -> Image as Numpy Array and Path of TFLite Model 
#### Output -> X and Y Coordinates

In [ ]:
image_as_np = np.asarray(image)

In [ ]:
def query_gmodel_np(image_nparray,model_path):
    interpreter = tf.lite.Interpreter(model_path)
    
    image_t = tf.convert_to_tensor(image_nparray)
    image_t = tf.expand_dims(image_t, 0)
    
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], test_tensor)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    return output_data    

In [ ]:
out_test_2 = query_gmodel_np(image_as_np,model_path)
print(out_test_2)

#### Function 
#### Input-> Image Path (For a file structure similar to Gazetrack Dataset) and Path of TFLite Model
#### Output-> Comparision by Visualization (Predicted vs Ground Truth Value)

In [ ]:
def comp_visualize(image_path,model_path):
    interpreter = tf.lite.Interpreter(model_path)
    
    image_t = tf.convert_to_tensor(image_nparray)
    image_t = tf.expand_dims(image_t, 0)
    
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], test_tensor)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    output_data = output_data.reshape(2,)
    gaze_gt = get_gt_gaze(image_path)
    
    plt.figure(figsize=(12,12))
    plt.grid(color='gray', linestyle='dashed')
    plt.scatter(0, 0, marker='*', s=200)
    plt.scatter(output_data[0], output_data[1], c='red', s=10)
    plt.scatter(gaze_gt[0], gaze_gt[1], c='blue', marker='+', s=200)
    plt.xlabel('distance from the camera at origin in x direction (cm)')
    plt.ylabel('distance from the camera at origin in y direction (cm)')
    plt.axis('scaled')
    plt.xlim(-2.5, 4)
    plt.ylim(-13, 0.2)

### Input Conversion to tf.example protocol buffer

In [ ]:
IMAGE_LOCATION = b'/Users/prakanshulsaxena/Downloads/gazetrack/train/images/00342__00074.jpg'
TFLITE_LOCATION = b'/Users/prakanshulsaxena/Downloads/model.tflite'

In [ ]:
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [ ]:
feature_test = _bytes_feature(b'test_string')
feature_test.SerializeToString()

In [ ]:
serialized_example = serialize_example(IMAGE_LOCATION,TFLITE_LOCATION)
serialized_example

In [ ]:
example_proto = tf.train.Example.FromString(serialized_example)
example_proto

#### Function
#### Input -> Image location 
#### -> tf.example protocol containing Image location and Image as a tensor 

#### Function
#### Input -> Image location and TFLite Model Location
#### -> tf.example protocol containing both the input parameters

### Visualization (Gazetrack Dataset)